<a href="https://colab.research.google.com/github/helincesxyz/finetuneLLM/blob/main/ConversationalChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Install Dependencies



To enable CUDA:
First, you need to ensure that your Colab notebook is set to use a GPU. Here's how to enable a GPU in Colab:

1. Go to the "Edit" menu and select "Notebook settings" or "Runtime" > "Change runtime type".
2. Choose "GPU" and T4 as the hardware accelerator.
3. Click "Save" to apply the changes.
You can also use Colab Pro and choose A100 or V100 and High RAM nfor faster use.

Please Note:
- Using isolated sandbox systems such as Google Colabs significantly lowers the possibility of conflicts or problems with permissions with the system package manager.

- Generally, without taking certain unusual procedures, **you are not able to construct classic virtual environments in Colab using tools like virtualenv or venv**. It is therefore often safe to proceed with pip installations directly in Colab notebooks
 *(Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the system package manager. It is recommended to use a virtual environment instead: https://pip.pypa.io/warnings/venv)*

- The -U flag ensures that if the library is already installed, it will be upgraded to the specified version to prevent any bugs.

In [1]:
%%bash
pip install -Uqqq pip
pip install -U bitsandbytes #supports cuda 12.2
pip install -U torch
pip install -qqq -U git+https://github.com/huggingface/transformers.git
pip install -qqq -U git+https://github.com/huggingface/peft.git
pip install -qqq -U git+https://github.com/huggingface/accelerate.git
pip install -U datasets
pip install -U loralib
pip install -q -U einops
pip install -q -U safetensors
pip install -q -U torch
pip install -q -U xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 111.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.2.2 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.2.2 which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have torch 2.2.2 which is incompatible.


In [ ]:
!nvidia-smi #check cuda

NVIDIA-SMI couldn't find libnvidia-ml.so library in your system. Please make sure that the NVIDIA Display Driver is properly installed and present in your system.
Please also try adding directory that contains libnvidia-ml.so to your system PATH.


#1.1 Import Installed Libraries

In [ ]:

!git clone https://github.com/TimDettmers/bitsandbytes.git


Cloning into 'bitsandbytes'...
remote: Enumerating objects: 6291, done.
remote: Counting objects: 100% (2469/2469), done.
remote: Compressing objects: 100% (359/359), done.
remote: Total 6291 (delta 2275), reused 2164 (delta 2100), pack-reused 3822
Receiving objects: 100% (6291/6291), 1.90 MiB | 9.14 MiB/s, done.
Resolving deltas: 100% (4246/4246), done.


1.
check to see if libraries are installed in the right place


In [ ]:
#check cuda lib
import os

# Check if the CUDA libraries directory exists
cuda_lib_dir = '/usr/local/cuda/lib64'
if os.path.exists(cuda_lib_dir):
    print("CUDA libraries directory found:", cuda_lib_dir)
else:
    print("CUDA libraries directory not found.")


CUDA libraries directory found: /usr/local/cuda/lib64


In [ ]:
os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda/lib64"


In [ ]:
"""
# Open and read the contents of utils.py
with open('/usr/local/lib/python3.10/dist-packages/bitsandbytes/utils.py', 'r') as f:
    print(f.read())
"""

In [ ]:
import bitsandbytes as bnb

In [ ]:
import json
import pandas as pd
import torch
import torch.nn as nn
import transformers
from pprint import pprint
from datasets import load_dataset
from accelerate import Accelerator
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

#1.2 Access HuggingFace API Token

In [ ]:
notebook_login()

#2. Upload Falcon 7B Model and QLora for FineTuning

We're going to import **Accelerate**: At Hugging Face, we created the 🤗 Accelerate library to help users easily train a 🤗 Transformers model on any type of distributed setup, whether it is multiple GPU's on one machine or multiple GPU's across several machines.

In [ ]:
from accelerate import Accelerator


https://vilsonrodrigues.medium.com/run-your-private-llm-falcon-7b-instruct-with-less-than-6gb-of-gpu-using-4-bit-quantization-ff1d4ffbabcc maybe try this too? #source

https://github.com/rohan-paul/LLM-FineTuning-Large-Language-Models/blob/main/Falcon-7B_FineTuning_with_PEFT_and_QLORA.ipynb #source

To upload our pre-trained model, you'll need WRITE access token from HuggingFace. You can also try more specialized models dependent on you liking.

In [ ]:
MODEL_NAME = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/6.70k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

https://huggingface.co/blog/4bit-transformers-bitsandbytes?source=post_page-----ff1d4ffbabcc--------------------------------

bitsandbytes article, set parameters according to this

In [ ]:
#analyze the current model's trainable parameters and total parameters
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

  # from pytroch

In [ ]:

model = prepare_model_for_kbit_training(model)
#FIX ERROR: You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


note:  QLoRA: 16 BIT works best with the hf bitsandbytes


In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainables%: 0.13058363808693696


#3. Deployment Tests

#4. LOG PRIVATE DATASET

dataset: https://huggingface.co/datasets/empathetic_dialogues
https://www.kaggle.com/datasets/kreeshrajani/3k-conversations-dataset-for-chatbot/data

PLUS KAGGLE HUMAN-ROBOT TEXT

In [ ]:
prompt = """
<human>: Hello, how are you?
<assistant>:
""".strip()



In [ ]:

#CONFIGURE TEMPERATURE and topp
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 1
generation_config.top_p = 1
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

https://github.com/curiousily/Get-Things-Done-with-Prompt-Engineering-and-LangChain/blob/master/07.falcon-qlora-fine-tuning.ipynb #source

In [ ]:
print(encoding.input_ids.shape)
print(encoding.attention_mask.shape)

torch.Size([1, 15])
torch.Size([1, 15])


In [ ]:
print(model.config.num_attention_heads)

71


In [ ]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )


print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on 

IndexError: tuple index out of range

In [ ]:
from datasets import load_dataset

data = load_dataset("csv", data_files={'Conversation.csv'})


In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'question', 'answer'],
        num_rows: 3725
    })
})

In [ ]:
#check dataset

data["train"][0]
# Inspect dataset structure and features
#print(data)
#data["train"].features


{'Unnamed: 0': 0,
 'question': 'hi, how are you doing?',
 'answer': "i'm fine. how about yourself?"}

In [ ]:
def generate_prompt(data_point):
  return f"""
<human>: {data_point["question"]}
<assistant>: {data_point["answer"]}
""".strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

In [ ]:
data = data["train"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/3725 [00:00<?, ? examples/s]

In [ ]:
data

Dataset({
    features: ['Unnamed: 0', 'question', 'answer', 'input_ids', 'attention_mask'],
    num_rows: 3725
})

#5. Fine-tuning The Model

In [ ]:
requires_grad=True

In [ ]:
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=1,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)


model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


RuntimeError: Inference tensors cannot be saved for backward. To work around you can make a clone to get a normal tensor and use it in autograd.

#6. Save And Deploy The Model

In [ ]:
model.save_pretrained("trained-model")

In [ ]:
PEFT_MODEL = "helincesxyz/convChatbot"

model.push_to_hub(
    PEFT_MODEL, token=True
)

In [ ]:
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

#6.1 Running The model

NEW PAPER FOR BETTER FINTEUNING:

In [ ]:

generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time
device = "cuda:0"

prompt = """
<human>: how have you been feeling lately?
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.no_grad():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


IndexError: tuple index out of range

#6.2 Deployment with Streamlit or FastAPI

In [ ]:
#not yet configured